In [1]:
import os

import glob
import torch
from uuid import uuid4
from huggingface_hub.file_download import http_get
from langchain_community.document_loaders import UnstructuredMarkdownLoader
#from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.docstore.document import Document
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim


SYSTEM_PROMPT = "Ты — очень умный ассистент-аналитик с системным мышлением. Виртуозно находишь связи в самых разных мыслях и идеях, используешь нестандартные подходы. Вот пример твоей задачи: Заметка 1: Сельское хозяйство сталкивается с серьёзными вызовами в связи с изменением климата. Участившиеся засухи, наводнения и вспышки вредителей непосредственно влияют на урожайность и производство продовольствия. Изменение климата приводит к тому, что традиционные агроклиматические зоны смещаются, заставляя фермеров адаптироваться к новым условиям, возможно, даже менять культуры, которые они выращивают. Эти изменения требуют значительных усилий и ресурсов для модернизации сельскохозяйственных технологий и практик, включая улучшенное управление водными ресурсами и внедрение устойчивых к изменению климата сортов растений. Заметка 2: Искусственный интеллект (ИИ) революционизирует медицину, предлагая новые способы диагностики и лечения заболеваний. Системы ИИ способны анализировать большие объёмы медицинских данных быстрее и точнее, чем это могут сделать врачи, что способствует более раннему выявлению заболеваний и повышению эффективности лечения. ИИ используется для анализа рентгеновских снимков, МРТ, для предсказания риска развития определённых состояний и даже для создания индивидуализированных планов лечения. Внедрение ИИ в медицине обещает сократить нагрузку на медицинский персонал, уменьшить ошибки и улучшить качество ухода за пациентами.. Сходство: Обе заметки подчёркивают необходимость адаптации и применения новых технологий для решения возникающих проблем: в сельском хозяйстве — это адаптация к изменению климата с помощью новых сортов растений и управления водными ресурсами, в медицине — использование искусственного интеллекта для улучшения диагностики и лечения. Таким образом, обе заметки иллюстрируют, как наука и технологии играют ключевую роль в адаптации и преобразовании отраслей для справления с вызовами современного мира. Заметка 1: "

LOADER_MAPPING = {
    ".md": (UnstructuredMarkdownLoader, {})
}

In [2]:
EMBEDDER = SentenceTransformer("intfloat/multilingual-e5-small")

In [3]:
from langchain_community.llms import LlamaCpp
#from langchain_community.callbacks.manager import call
#from langchain_community.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# Callbacks support token-wise streaming
#callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
MODEL = LlamaCpp(
    model_path="C:/Users/user/Desktop/llm_models/mistral-7b-openorca.Q3_K_M.gguf",
    top_p=1,
    n_ctx = 5500,
 #   callback_manager=callback_manager,
    verbose=True  # Verbose is required to pass to the callback manager
)

llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from C:/Users/user/Desktop/llm_models/mistral-7b-openorca.Q3_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = open-orca_mistral-7b-openorca
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:    

In [4]:
def get_uuid():
    return str(uuid4())

def load_single_document(file_path: str) -> Document:
    ext = "." + file_path.rsplit(".", 1)[-1]
    assert ext in LOADER_MAPPING
    loader_class, loader_args = LOADER_MAPPING[ext]
    loader = loader_class(file_path, **loader_args)
    doc_output = loader.load()[0]
   # setattr(doc_output, 'file_name', file_path.rsplit(".", 1)[0].split("/")[-1].split("\\")[-1])
    return doc_output

def process_text(text):
    lines = text.split("\n")
    lines = [line for line in lines if len(line.strip()) > 2]
    text = "\n".join(lines).strip()
    if len(text) < 10:
        return None
    return text

def upload_files(files, file_paths):
    file_paths = [f.name for f in files]
    return file_paths

def build_index(file_paths, db= None, chunk_size= None, chunk_overlap= None, file_warning=None):
    documents = [load_single_document(path) for path in file_paths]
   # text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    #documents = text_splitter.split_documents(documents)
    #print("Documents after split:", len(documents))
    fixed_documents = []
    for doc in documents:
        doc.page_content = process_text(doc.page_content)
        if not doc.page_content:
            continue
        fixed_documents.append(doc)
    print("Documents after processing:", len(fixed_documents))

    texts = [doc.page_content for doc in fixed_documents]
    names = [doc.metadata['source'].rsplit(".", 1)[0].split("/")[-1].split("\\")[-1] for doc in fixed_documents]
    embeddings = EMBEDDER.encode(texts, convert_to_tensor=True)
    db = {"name_list": names,"docs": texts, "embeddings": embeddings}
    print("Embeddings calculated!")
    return db

def find_md_files(folder_path):
    files = []  # List to hold the paths of .md files

    # Function to recursively search the directory
    def search_directory(directory):
        # List all files and directories in the current directory
        for entry in os.listdir(directory):
            full_path = os.path.join(directory, entry)
            # Check if the current entry is a directory
            if os.path.isdir(full_path):
                search_directory(full_path)  # Recursively search this directory
            elif full_path.endswith('.md'):
                files.append(full_path)  # Add the .md file to the list

    search_directory(folder_path)
    return files


In [5]:
def retrieve(user_message, name, db, k_documents):
    retrieved_docs = ""
    if db:
        query_embedding = EMBEDDER.encode(user_message, convert_to_tensor=True)
        scores = cos_sim(query_embedding, db["embeddings"])[0]
        exclude_index = db['name_list'].index(name) if name in db['name_list'] else None
        
        # Если есть индекс для исключения, устанавливаем его значение сходства на минимум, чтобы исключить из topk
        if exclude_index is not None:
            scores[exclude_index] = scores.min() - 1  # Устанавливаем значение ниже минимального, чтобы исключить документ
        
        top_k_idx = torch.topk(scores, k=k_documents)[1]
        top_k_documents = [db["docs"][idx] for idx in top_k_idx]
        top_k_documents_name = [db["name_list"][idx] for idx in top_k_idx]
        
    return top_k_documents, top_k_documents_name

def find_latest_md_file(start_path):
    # Ищем все файлы с расширением .md в указанной папке и её подпапках
    md_files = glob.glob(start_path + '/**/*.md', recursive=True)
    
    # Если файлы найдены, определяем последний измененный файл
    if md_files:
        latest_file = max(md_files, key=os.path.getmtime)
        return latest_file
    else:
        return "Файлы с расширением .md не найдены."
    
def find_file(filename, root_folder):
    for root, dirs, files in os.walk(root_folder):
        if filename+'.md' in files:
            return os.path.join(root, filename+'.md')
    return None

In [6]:
file_paths=find_md_files("C:/Users/user/PolinasMind")
db = build_index(file_paths)

Documents after processing: 26
Embeddings calculated!


In [ ]:
input_message = input()

if input_message=='last' or input_message=='l':
    file_path = find_latest_md_file("C:/Users/user/PolinasMind")
elif input_message=='help':
    print("напиши 'last' или 'l' и тогда я найду заметки, похожие на последнюю добавленную. А затем напишу чем именно они похожи. Или напиши название своей заметки, к которой хочешь найти похожие")
else: 
    file_path = find_file(input_message, "C:/Users/user/PolinasMind")

print(file_path)
if file_path:
    document = load_single_document(file_path)
    name = file_path.rsplit(".", 1)[0].split("/")[-1].split("\\")[-1]
    user_message = process_text(document.page_content)
    retrieved_documents, names_of_retrieved_docs = retrieve(user_message, name, db, 1)
    
    for i,note in enumerate(retrieved_documents):
        print("NOTE TO COMPARE NAME", names_of_retrieved_docs[i])
        output = MODEL.invoke(SYSTEM_PROMPT+str(user_message)+" Заметка 2: "+note+" Сходство: ")
        print("THEIR SIMILARITY IN", output)
else:
    print('I cant understant what do you want')
        

C:/Users/user/PolinasMind\Resources\ML\Моделирование AGI с помощью физики.md
NOTE TO COMPARE NAME Теория реляционных фреймов - саммари
